# Import

## Import Files

In [ ]:
!pip install gdown

In [ ]:
import gdown
import os

file_id = '1_Xo2vU82JSSadBdD1Kb7iImHnEYdoFGh'
output = 'open.zip' # 저장할 파일 이름

# 'output'으로 지정된 파일이 현재 경로에 존재하지 않을 경우에만 다운로드 실행
if not os.path.exists(output):
    print(f"'{output}' 파일이 없어 다운로드를 시작합니다.")
    gdown.download(id=file_id, output=output)
else:
    print(f"'{output}' 파일이 이미 존재합니다. 다운로드를 건너뜁니다.")

'open.zip' 파일이 이미 존재합니다. 다운로드를 건너뜁니다.


In [ ]:
# !unzip -qq '/파일 경로/파일명.zip' -d '저장할 dir 위치 경로'
!unzip -qq '/content/open.zip' -d '/content/'

replace /content/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


## Baseline

In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm
import holidays
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV

# Fixed RandomSeed & Setting Hyperparameter

## Baseline

In [ ]:
def set_seed(seed=42):
    random.seed(seed)    # 1. 파이썬 내장 random 라이브러리의 시드를 고정합니다.
    np.random.seed(seed)    # 2. NumPy 라이브러리의 난수 생성 시드를 고정합니다.
    torch.manual_seed(seed)    # 3. PyTorch의 CPU 연산에 대한 난수 생성 시드를 고정합니다.
    os.environ['PYTHONHASHSEED'] = str(seed)    # 4. 파이썬의 해시 시드를 고정하여 딕셔너리 등의 순서를 보장합니다.

    # 5. CUDA (GPU) 사용이 가능한 경우, GPU 관련 시드 및 설정을 고정합니다.
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)    # 5-1. 현재 사용 중인 GPU의 난수 생성 시드를 고정합니다.
        torch.cuda.manual_seed_all(seed)    # 5-2. 여러 개의 GPU를 사용하는 경우, 모든 GPU의 시드를 고정합니다.
        torch.backends.cudnn.deterministic = True   # 5-3. cuDNN 라이브러리가 항상 결정적인(deterministic) 알고리즘만 사용하도록 설정합니다.
        torch.backends.cudnn.benchmark = False  # 5-4. cuDNN의 벤치마크 기능을 비활성화합니다.

set_seed(42)    # 위에서 정의한 함수를 seed 값 42로 실행하여 코드 전체의 재현성을 확보합니다.

In [ ]:
"""
LOOKBACK=28: 과거 28일치 데이터를 보고
PREDICT=7: 미래 7일치를 에측
BATCH_SIZE=16: 한번에 16개의 data를 묶어 학습
EPOCHS=50: 전체 데이터를 50번 반복학습
"""
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Load

In [ ]:
train = pd.read_csv('./train/train.csv')

In [ ]:
import numpy as np
import pandas as pd
import holidays
import re

def dataPreProcessing(df):
    # 1. 열 이름 정리
    df = df.rename(columns={
        '영업일자': 'date_time',
        '영업장명_메뉴명': 'market_menu',
        '매출수량': 'sales_amount'
    }) if '영업일자' in df.columns else df

    # 2. 날짜 파생
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['weekday'] = df['date_time'].dt.weekday
    df['weekend'] = df['weekday'].isin([5, 6]).astype(int)
    df['month'] = df['date_time'].dt.month
    df['day'] = df['date_time'].dt.day

    # <<< ADDED: 추가 시간 관련 피처
    df['day_of_year'] = df['date_time'].dt.dayofyear
    df['week_of_year'] = df['date_time'].dt.isocalendar().week.astype(int)
    df['quarter'] = df['date_time'].dt.quarter

    # <<< ADDED: 주기성을 나타내는 sin/cos 변환 피처
    df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
    df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
    df['day_sin'] = np.sin(2 * np.pi * df['day']/31)
    df['day_cos'] = np.cos(2 * np.pi * df['day']/31)
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday']/7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday']/7)

    # 3. 공휴일
    kr_holidays = holidays.KR()
    df['holiday'] = df['date_time'].dt.date.apply(lambda d: int(d in kr_holidays))

    # 4. 방학 시즌 (1,2,7,8월)
    df['vacation_season'] = df['month'].isin([1, 2, 7, 8]).astype(int)

    # 5. 메뉴명 분해
    df[['store_name', 'menu_name']] = df['market_menu'].str.split('_', n=1, expand=True)

    # 6. 사용유형 추정
    def classify_usage_type(menu):
        if pd.isna(menu): return '기타'
        if '어린이' in menu: return '어린이'
        elif re.search(r'단체|플래터|무제한|[3-9]인|인분|세트', str(menu)): return '단체'
        elif '2인' in menu: return '커플'
        elif '1인' in menu or '단품' in menu or 'Gls' in menu: return '1인'
        else: return '일반'
    df['usage_type'] = df['menu_name'].apply(classify_usage_type)

    # 7. 고급 메뉴 여부
    df['is_premium'] = df['menu_name'].str.contains('한우|프리미엄|수제|특선|와인').fillna(False).astype(int)

    # 10. 라벨 인코딩 (사용유형)
    usage_map = {'1인': 0, '커플': 1, '단체': 2, '어린이': 3, '일반': 4, '기타': 5}
    df['usage_type_encoded'] = df['usage_type'].map(usage_map).fillna(5).astype(int)

    # <<< ADDED: 상호작용 피처 (가게 + 요일)
    df['store_name_encoded'] = df['store_name'].astype('category').cat.codes
    df['store_weekday_encoded'] = (df['store_name'].astype(str) + '_' + df['weekday'].astype(str)).astype('category').cat.codes

    return df


In [ ]:
def add_ts_features(df): # <<< CHANGED: 함수 전체 수정
    df = df.sort_values(['market_menu','date_time']).copy()
    g = df.groupby('market_menu')['sales_amount']

    # <<< CHANGED: Lag 기간 확장
    lags = [1, 2, 3, 7, 14, 21, 28]
    for lag in lags:
        df[f'lag_{lag}'] = g.shift(lag)

    # <<< CHANGED: Rolling 윈도우 및 통계량 확장
    roll_windows = [7, 14, 28]
    for window in roll_windows:
        shifted = g.shift(1) # 현재 시점 정보가 들어가지 않도록 shift(1)
        df[f'rolling_mean_{window}'] = shifted.rolling(window, min_periods=1).mean()
        df[f'rolling_std_{window}'] = shifted.rolling(window, min_periods=1).std()
        df[f'rolling_min_{window}'] = shifted.rolling(window, min_periods=1).min()
        df[f'rolling_max_{window}'] = shifted.rolling(window, min_periods=1).max()

    num_cols = [c for c in df.columns if df[c].dtype.kind in 'if']
    df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan).fillna(0)
    return df

In [ ]:

def smape(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    out = np.zeros_like(denom)
    mask = denom > 0
    out[mask] = np.abs(y_true[mask] - y_pred[mask]) / denom[mask]
    return out.mean() * 100.0

def add_days_since_last_sale(df):
    # 정렬 & 라벨 안전화
    df = df.sort_values(['market_menu','date_time']).copy()
    df['sales_amount'] = pd.to_numeric(df['sales_amount'], errors='coerce').fillna(0).clip(lower=0)

    # 그룹별 '마지막 양수 판매일'을 forward-fill로 계산
    pos_date = df['date_time'].where(df['sales_amount'] > 0)
    df['last_pos_date'] = pos_date.groupby(df['market_menu']).ffill()

    # 일수 계산(벡터화): pandas Timedelta → .dt.days 사용
    delta = df['date_time'] - df['last_pos_date']
    df['days_since_last_sale'] = delta.dt.days.fillna(999).astype(int)

    # 보조 컬럼 정리
    df = df.drop(columns=['last_pos_date'])
    return df



In [ ]:

train_df = dataPreProcessing(train)
train_feat = add_ts_features(train_df)

train_feat = add_days_since_last_sale(train_feat)
train_df

,date_time,market_menu,sales_amount,weekday,weekend,month,day,day_of_year,week_of_year,quarter,...,weekday_cos,holiday,vacation_season,store_name,menu_name,usage_type,is_premium,usage_type_encoded,store_name_encoded,store_weekday_encoded
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,6,1,1,1,1,52,1,...,0.623490,1,1,느티나무 셀프BBQ,1인 수저세트,단체,0,2,0,6
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,0,0,1,2,2,1,1,...,1.000000,0,1,느티나무 셀프BBQ,1인 수저세트,단체,0,2,0,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,1,0,1,3,3,1,1,...,0.623490,0,1,느티나무 셀프BBQ,1인 수저세트,단체,0,2,0,1
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,2,0,1,4,4,1,1,...,-0.222521,0,1,느티나무 셀프BBQ,1인 수저세트,단체,0,2,0,2
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,3,0,1,5,5,1,1,...,-0.900969,0,1,느티나무 셀프BBQ,1인 수저세트,단체,0,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12,1,0,6,11,163,24,2,...,0.623490,0,0,화담숲카페,현미뻥스크림,일반,0,4,8,57
102672,2024-06-12,화담숲카페_현미뻥스크림,10,2,0,6,12,164,24,2,...,-0.222521,0,0,화담숲카페,현미뻥스크림,일반,0,4,8,58
102673,2024-06-13,화담숲카페_현미뻥스크림,14,3,0,6,13,165,24,2,...,-0.900969,0,0,화담숲카페,현미뻥스크림,일반,0,4,8,59
102674,2024-06-14,화담숲카페_현미뻥스크림,12,4,0,6,14,166,24,2,...,-0.900969,0,0,화담숲카페,현미뻥스크림,일반,0,4,8,60


In [ ]:
train_df['sales_amount'] = (
    pd.to_numeric(train_df['sales_amount'], errors='coerce')
      .fillna(0)
      .clip(lower=0)
)

In [ ]:
feature_cols = [
    # 원본 모델의 중요 피처
    'weekend', 'holiday', 'vacation_season', 'usage_type_encoded', 'is_premium',

    # 이번 분석에서 가장 중요하게 확인된 피처
    'days_since_last_sale',

    # 중요도가 매우 높았던 확장된 Lag/Rolling 피처들
    'lag_1', 'lag_2', 'lag_3', 'lag_7', 'lag_14', 'lag_21', 'lag_28',
    'rolling_mean_7', 'rolling_std_7', 'rolling_min_7', 'rolling_max_7',
    'rolling_mean_14', 'rolling_std_14',
    'rolling_mean_28', 'rolling_std_28',
]






# Define Model

## Baseline

# Train

## Baseline

# Prediction

## Baseline

## gagyeomkim

## KGY

In [ ]:
def build_test_features(train_df, raw_test_df, lags=(1,7,14), roll_window=7):
    # 1) 공통 전처리(한글→영문 리네임, 캘린더 파생 등)
    T = dataPreProcessing(raw_test_df).copy()

    # 2) test 28일에 실제 매출이 있으면 그대로 사용 (없으면 0으로 대체)
    if 'sales_amount' not in T.columns:
        T['sales_amount'] = 0.0

    combo = T.sort_values(['market_menu','date_time'])
    combo = add_ts_features(combo, lags=lags, roll_window=roll_window)
    combo = add_days_since_last_sale(combo)

    feat_test = combo[combo['date_time'].isin(T['date_time']) &
                      (combo['market_menu'].isin(T['market_menu']))].copy()
    return feat_test



In [ ]:
def forecast_next7_for_block(
    train_df,
    raw_test_df,
    feature_cols,
    clf,
    reg,
    tau_by_menu=None,          # hard 모드에서만 사용
    default_tau=0.35,          # hard 모드의 기본 τ (off는 <0), soft 모드에선 무시
    block_id="TEST_XX",
    gate_mode="soft",          # "soft"(기본) | "hard" | "off"
    alpha=0.30,
    prob_gamma=0.6,            # soft 게이팅 지수: y_final = y_hat * p_pos**alpha
    tau_floor_prob=0.0,       # p_pos가 이보다 작으면 floor 미적용
    floor_ratio=0.12,          # floor = (컨텍스트 양수 평균)*floor_ratio
    scale_clip=(0.8, 12.0),     # 컨텍스트 스케일 클리핑 범위
    apply_scale=True,
    apply_floor=True
):
    """
    반환: DataFrame[token, 영업장명_메뉴명, 매출수량]  (+1~+7일)
    전제: dataPreProcessing, add_ts_features, add_days_since_last_sale가 이미 정의되어 있음.
    """

    # 0) 컨텍스트 28일 준비
    T = dataPreProcessing(raw_test_df).copy()
    if 'sales_amount' not in T.columns:
        T['sales_amount'] = 0.0

     # 컨텍스트만으로 파생
    combo = T.sort_values(['market_menu','date_time']).copy()
    combo = add_ts_features(combo)
    combo = add_days_since_last_sale(combo)

    preds = []
    ctx_last = combo['date_time'].max()
    ctx_start = ctx_last - pd.Timedelta(days=27)   # 28일 창
    menus = T['market_menu'].drop_duplicates().tolist()

    # 1) 컨텍스트 기반 보정량 계산 (soft 모드용)
    #    - 메뉴별 floor(양수 평균*ratio)
    #    - 최근 7일 실제합/예측합 비율로 scale
    floor_map, scale_by_menu = {}, {}
    if gate_mode == "soft":
        # (A) floor
        ctx_hist = combo[(combo['date_time'] >= ctx_start) & (combo['date_time'] <= ctx_last)].copy()
        q = 0.45
        pos_q = (ctx_hist[ctx_hist['sales_amount'] > 0]
             .groupby('market_menu')['sales_amount']
             .quantile(q).fillna(0.0))
        floor_map = (floor_ratio * pos_q).to_dict() if apply_floor else {}

        # (B) scale (최근 7일)
        ctx7_mask = combo['date_time'].between(ctx_last - pd.Timedelta(days=6), ctx_last)
        scale_by_menu = {}
        if ctx7_mask.any():
            X_ctx7 = (ctx_hist.loc[ctx7_mask, feature_cols]
                  .apply(pd.to_numeric, errors='coerce').fillna(0).astype(float))
            y_ctx7_hat = np.expm1(reg.predict(X_ctx7))
            y_ctx7_hat = np.nan_to_num(y_ctx7_hat, nan=0.0, posinf=0.0, neginf=0.0).clip(0)

            hat_sum_by_menu = pd.Series(y_ctx7_hat,
                                    index=ctx_hist.loc[ctx7_mask, 'market_menu']).groupby(level=0).sum()
            true_sum_by_menu = ctx_hist.loc[ctx7_mask].groupby('market_menu')['sales_amount'].sum()
            scale_series = (true_sum_by_menu / (hat_sum_by_menu + 1e-6)).fillna(1.0)
            global_scale = float(true_sum_by_menu.sum() / (hat_sum_by_menu.sum() + 1e-6))
            scale_series = 0.5*scale_series + 0.5*global_scale
            if apply_scale:
                scale_series = scale_series.clip(lower=0.8, upper=12.0)
                scale_by_menu = scale_series.to_dict()
            else:
                scale_by_menu = {k: 1.0 for k in true_sum_by_menu.index}

    # 2) 7-step 롤아웃
    for h in range(1, 8):
        fut_date = ctx_last + pd.Timedelta(days=h)
        # 미래 한 줄(각 메뉴) 생성 + 공통 파생
        fut = pd.DataFrame({
            'date_time': [fut_date]*len(menus),
            'market_menu': menus,
            'sales_amount': 0.0
        })
        fut = dataPreProcessing(fut)[[
            'date_time','market_menu','sales_amount',
            'weekend','holiday','vacation_season',
            'usage_type_encoded','is_premium','store_name','menu_name'
        ]]

        # 컨테이너에 붙여 동일 피처 생성
        combo = pd.concat([combo, fut], ignore_index=True, sort=False)
        combo = combo.sort_values(['market_menu','date_time'])
        combo = add_ts_features(combo)
        combo = add_days_since_last_sale(combo)

        # 방금 추가한 미래행만 예측
        mask = (combo['date_time'] == fut_date)
        Xf = combo.loc[mask, feature_cols].copy()
        Xf = Xf.apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

        p_pos = clf.predict_proba(Xf)[:, 1]
        # 선택: 확률 팽창(작을수록 덜 깎이게 함)
        if prob_gamma is not None and prob_gamma != 1.0:
            p_pos = 1.0 - np.power(1.0 - p_pos, prob_gamma)
        y_hat = np.expm1(reg.predict(Xf))
        y_hat = np.nan_to_num(y_hat, nan=0.0, posinf=0.0, neginf=0.0).clip(0)

        if gate_mode == "off" or (default_tau is not None and default_tau < 0):
            # 게이팅 완전 OFF
            y_final = y_hat.copy()

        elif gate_mode == "hard":
            # 하드 게이팅 (기존 방식 유지 옵션)
            if tau_by_menu is None:
                tau_vec = np.full(len(Xf), (default_tau if default_tau is not None else 0.35))
            else:
                tau_vec = np.array([
                    tau_by_menu.get(m, (default_tau if default_tau is not None else 0.35))
                    for m in combo.loc[mask, 'market_menu']
                ])
            y_final = np.where(p_pos < tau_vec, 0.0, y_hat).clip(0)

        else:
            # SOFT 게이팅 + 스케일 + 바닥값
            # 1) 소프트 게이팅
            y_soft = y_hat * np.power(p_pos, (alpha if alpha is not None else 0.0))

            # 2) 메뉴별 스케일
            if apply_scale and len(scale_by_menu):
                s_vec = np.array([scale_by_menu.get(m, 1.0) for m in combo.loc[mask, 'market_menu']])
                y_scaled = (y_soft * s_vec).clip(0)
            else:
                y_scaled = y_soft

            # 3) 바닥값 (확률이 너무 낮으면 floor 미적용)
            if apply_floor and len(floor_map):
                floor_vec = np.array([floor_map.get(m, 0.0) for m in combo.loc[mask, 'market_menu']])
                apply_mask = p_pos >= (tau_floor_prob if tau_floor_prob is not None else 0.0)
                # p_pos가 너무 낮으면 floor 미적용 → 그대로 y_scaled
                y_final = np.where(apply_mask, np.maximum(y_scaled, floor_vec), y_scaled).clip(0)
            else:
                y_final = y_scaled

        ctx_hist = combo[combo['date_time'] <= ctx_last].copy()  # 컨텍스트 28일까지만
        dow = fut_date.dayofweek
        true_day_sums = (
            ctx_hist.assign(dow=ctx_hist['date_time'].dt.dayofweek)
                    .query('dow == @dow')
                    .groupby('date_time')['sales_amount'].sum()
        )
        if len(true_day_sums):
            target_sum = float(true_day_sums.median())  # 필요시 mean()으로 교체 가능
            pred_sum = float(y_final.sum()) + 1e-6
            k = np.clip(target_sum / pred_sum, 0.5, 8.0)  # 과도 왜곡 방지
            y_final = (y_final * k).clip(0)

        # 다음 step의 라그/롤링 반영을 위해 예측값을 combo에 기록
        combo.loc[mask, 'sales_amount'] = y_final

        # 결과 저장
        preds.append(pd.DataFrame({
            'token': [f"{block_id}+{h}일"] * np.sum(mask),
            '영업장명_메뉴명': combo.loc[mask, 'market_menu'].values,
            '매출수량': y_final
        }))

    return pd.concat(preds, ignore_index=True)


In [ ]:
!pip install optuna

In [ ]:
import optuna
from xgboost import XGBClassifier, XGBRegressor
from sklearn.calibration import CalibratedClassifierCV
import numpy as np
import pandas as pd

# <<< ADDED: Optuna를 위한 새로운 CV 함수
# <<< CHANGED: CV 대신 한 번만 빠르게 검증하는 새 함수로 교체

# <<< FIXED: 오류가 수정된 빠른 검증 함수

def quick_validate_for_optuna(
    df_prepped,
    feature_cols,
    clf_params,
    reg_params,
    post_params
):
    df_prepped = df_prepped.sort_values(['market_menu','date_time']).copy()
    uniq = np.array(sorted(df_prepped['date_time'].unique()))

    ctx_days, horizon = 28, 7
    val_start = uniq[-(horizon)]
    ctx_start = uniq[-(horizon + ctx_days)]
    train_part = df_prepped[df_prepped['date_time'] < ctx_start].copy()
    val_part = df_prepped[df_prepped['date_time'] >= ctx_start].copy()

    tr_feat = add_ts_features(train_part); tr_feat = add_days_since_last_sale(tr_feat)
    y = pd.to_numeric(tr_feat['sales_amount'], errors='coerce').fillna(0).clip(lower=0).to_numpy()
    X_tr = tr_feat[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

    pos_rate = (y > 0).mean(); neg_pos  = (1 - pos_rate) / max(pos_rate, 1e-6)
    clf = XGBClassifier(**clf_params, random_state=42, n_jobs=-1, scale_pos_weight=neg_pos)
    clf.fit(X_tr, (y > 0).astype(int))

    w = 1.0 / (1.0 + y); t = np.log1p(y)
    reg = XGBRegressor(**reg_params, random_state=42, n_jobs=-1)
    reg.fit(X_tr, t, sample_weight=w)

    combo = val_part.copy()
    val_truth = val_part[val_part['date_time'] >= val_start][['market_menu','date_time','sales_amount']].copy()
    initial_menus = sorted(val_truth['market_menu'].unique())

    rows = []
    for h in range(horizon):
        fut_date = val_start + pd.Timedelta(days=h)
        fut = pd.DataFrame({'date_time': [fut_date]*len(initial_menus), 'market_menu': initial_menus, 'sales_amount': 0.0})
        fut = dataPreProcessing(fut)

        combo_with_fut = pd.concat([combo, fut], ignore_index=True, sort=False).sort_values(['market_menu','date_time'])
        combo_with_fut = add_ts_features(combo_with_fut)
        combo_with_fut = add_days_since_last_sale(combo_with_fut)

        mask = combo_with_fut['date_time'].eq(fut_date)
        Xf = combo_with_fut.loc[mask, feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)

        p_pos = clf.predict_proba(Xf)[:, 1]
        y_hat = np.expm1(reg.predict(Xf)).clip(0)

        y_final = (y_hat * np.power(p_pos, post_params['alpha'])).clip(0)

        # <<< FIXED: 예측을 수행한 메뉴 리스트를 바로 가져와서 사용
        menus_for_this_pred = combo_with_fut.loc[mask, 'market_menu'].values
        rows.append(pd.DataFrame({'market_menu': menus_for_this_pred, 'date_time': fut_date, 'pred': y_final}))

        temp_pred_row = fut.copy()
        # <<< FIXED: 길이가 다를 수 있으므로, 예측된 메뉴에 대해서만 값을 채움
        pred_map = dict(zip(menus_for_this_pred, y_final))
        temp_pred_row['sales_amount'] = temp_pred_row['market_menu'].map(pred_map).fillna(0)
        combo = pd.concat([combo, temp_pred_row], ignore_index=True)

    preds_df = pd.concat(rows, ignore_index=True)
    merged = val_truth.merge(preds_df, on=['market_menu','date_time'], how='left').fillna(0.0)

    return smape(merged['sales_amount'].to_numpy(), merged['pred'].to_numpy())

In [ ]:
# <<< ADDED: Optuna의 objective 함수 정의

def objective(trial):
    # 1. 탐색할 하이퍼파라미터 범위 정의
    # 분류기 파라미터
    clf_params = {
        'n_estimators': trial.suggest_int('clf_n_estimators', 400, 1000, step=100),
        'max_depth': trial.suggest_int('clf_max_depth', 5, 10),
        'learning_rate': trial.suggest_float('clf_learning_rate', 0.01, 0.1, log=True),
        'subsample': trial.suggest_float('clf_subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('clf_colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('clf_gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('clf_reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('clf_reg_lambda', 1e-8, 1.0, log=True),
    }

    # 회귀기 파라미터
    reg_params = {
        'n_estimators': trial.suggest_int('reg_n_estimators', 600, 1500, step=100),
        'max_depth': trial.suggest_int('reg_max_depth', 6, 12),
        'learning_rate': trial.suggest_float('reg_learning_rate', 0.01, 0.08, log=True),
        'subsample': trial.suggest_float('reg_subsample', 0.7, 1.0),
        'colsample_bytree': trial.suggest_float('reg_colsample_bytree', 0.7, 1.0),
        'gamma': trial.suggest_float('reg_gamma', 1e-8, 1.0, log=True),
        'reg_alpha': trial.suggest_float('reg_reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_reg_lambda', 1e-8, 1.0, log=True),
    }

    # 후처리 파라미터
    post_params = {
        'alpha': trial.suggest_float('alpha', 0.2, 0.8)
    }

    # 2. CV 함수를 호출하여 SMAPE 점수 계산
    # train_df는 dataPreProcessing이 완료된 데이터프레임입니다.
    # feature_cols는 이전에 정의한 핵심 피처 목록입니다.
    avg_smape = quick_validate_for_optuna(train_df, feature_cols, clf_params, reg_params, post_params)


    # 3. Optuna가 최소화할 점수(SMAPE)를 반환
    return avg_smape

In [ ]:
# <<< ADDED: Optuna 스터디 생성 및 실행

# 이전에 전처리/피처생성까지 완료된 train_df와 feature_cols를 사용합니다.
# train_df = dataPreProcessing(train)
# feature_cols = [ ... 핵심 피처 목록 ... ]

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10, timeout=3000) # 50회 시도

# 최적화 과정 시각화 (옵션)
# optuna.visualization.plot_optimization_history(study)
# optuna.visualization.plot_param_importances(study)

print("Best trial:")
trial = study.best_trial
print(f"  Value (SMAPE): {trial.value}")
print("  Best Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-08-24 17:05:46,619] A new study created in memory with name: no-name-8ce5eb04-d295-40c6-b576-a652c8a39d1b
[I 2025-08-24 17:05:59,960] Trial 0 finished with value: 57.973686240530796 and parameters: {'clf_n_estimators': 500, 'clf_max_depth': 5, 'clf_learning_rate': 0.03129878430378294, 'clf_subsample': 0.7810215357197617, 'clf_colsample_bytree': 0.8968735455013725, 'clf_gamma': 0.00016752532246859013, 'clf_reg_alpha': 1.665568052091143e-05, 'clf_reg_lambda': 3.86309007453106e-08, 'reg_n_estimators': 1000, 'reg_max_depth': 6, 'reg_learning_rate': 0.04670150962311326, 'reg_subsample': 0.7167900736907388, 'reg_colsample_bytree': 0.7597287258336951, 'reg_gamma': 8.868041082229822e-06, 'reg_reg_alpha': 0.11912990403005189, 'reg_reg_lambda': 1.410469387136265e-07, 'alpha': 0.35106493761296564}. Best is trial 0 with value: 57.973686240530796.
[I 2025-08-24 17:06:17,979] Trial 1 finished with value: 55.84022720654175 and parameters: {'clf_n_estimators': 800, 'clf_max_depth': 6, 'clf_lea

Best trial:
  Value (SMAPE): 54.88151670189336
  Best Params: 
    clf_n_estimators: 800
    clf_max_depth: 5
    clf_learning_rate: 0.018376781366573083
    clf_subsample: 0.9251387887424868
    clf_colsample_bytree: 0.955162118540763
    clf_gamma: 2.5644649197198383e-07
    clf_reg_alpha: 2.2930715255806882e-07
    clf_reg_lambda: 5.042914223101997e-07
    reg_n_estimators: 800
    reg_max_depth: 9
    reg_learning_rate: 0.014667811818787053
    reg_subsample: 0.7854627818011901
    reg_colsample_bytree: 0.9008333604932444
    reg_gamma: 8.590814208669663e-08
    reg_reg_alpha: 0.0002519579153147406
    reg_reg_lambda: 1.936099682313257e-06
    alpha: 0.5430655517529797


# Submission

## Baseline

### KGY VERSION

In [ ]:
# ------------------------------------------------------
# A. 블록별 예측 → full_pred_df (중복 없이 하나로)
# ------------------------------------------------------
def infer_all_blocks(test_glob_pattern,
                     train_df, feature_cols, clf, reg,
                     gate_kwargs):
    """
    test_glob_pattern: './test/TEST_*.csv'
    gate_kwargs: forecast_next7_for_block에 바로 전달할 게이팅/스케일 kwargs
                 (예: {'gate_mode':'soft', 'alpha':0.5, ...})
    반환: full_pred_df (token, 영업장명_메뉴명, 매출수량 포함)
    """
    test_files = sorted(glob.glob(test_glob_pattern))
    all_preds = []

    # tau_by_menu가 있으면 자동 첨부 (있을 때만)
    if 'tau_by_menu' not in gate_kwargs:
        try:
            gate_kwargs = {**gate_kwargs, 'tau_by_menu': tau_by_menu}
        except NameError:
            pass  # 사용 안 함

    for path in test_files:
        raw_test = pd.read_csv(path)
        block_id = os.path.splitext(os.path.basename(path))[0]  # e.g., TEST_00

        pred7 = forecast_next7_for_block(
            train_df=train_df,
            raw_test_df=raw_test,
            feature_cols=feature_cols,
            clf=clf, reg=reg,
            block_id=block_id,
            **gate_kwargs
        )
        all_preds.append(pred7)

    full_pred_df = pd.concat(all_preds, ignore_index=True)
    return full_pred_df


# ------------------------------------------------------
# B. full_pred_df → 제출 DF (벡터화, 조각화 경고 없음)
# ------------------------------------------------------
# <<< MODIFIED: round() 함수가 적용된 최종 제출 함수

def build_submission(full_pred_df, sample_submission_path, save_path=None):
    sample_submission = pd.read_csv(sample_submission_path)
    sub_tokens = sample_submission['영업일자'].astype(str)
    menu_cols = [c for c in sample_submission.columns if c != '영업일자']

    # dedup + pivot_table (기존과 동일)
    dedup = (full_pred_df
             .groupby(['token','영업장명_메뉴명'], as_index=False)['매출수량']
             .sum())
    wide_pred = dedup.pivot_table(index='token',
                                  columns='영업장명_메뉴명',
                                  values='매출수량',
                                  aggfunc='sum',
                                  fill_value=0.0)

    # 순서/컬럼 맞춰 한 번에 생성 (기존과 동일)
    arr = (wide_pred
           .reindex(index=sub_tokens, columns=menu_cols, fill_value=0.0)
           .astype('float64'))
    sub_out = arr.reset_index().rename(columns={'token':'영업일자'})

    # =======================================================
    # <<< ADDED: 모든 예측값 반올림 및 정수 변환
    # =======================================================
    # '영업일자'를 제외한 모든 메뉴 컬럼에 대해 round() 함수를 적용하고, 타입을 정수(int)로 바꿉니다.
    sub_out[menu_cols] = sub_out[menu_cols].round().astype(int)
    # =======================================================

    vals = sub_out[menu_cols].to_numpy(dtype='float64')
    print("[SUB] sum:", float(np.nansum(vals)))
    print("[SUB] nonzero:", int(np.count_nonzero(vals)))
    print("[SUB] zero_ratio:", float((vals == 0).mean()))

    if save_path is not None:
        os.makedirs(os.path.dirname(save_path) or '.', exist_ok=True)
        # <<< CHANGED: 정수로 저장하므로 float_format 옵션 제거
        sub_out.to_csv(save_path, index=False, encoding='utf-8-sig')
        print("💾 Saved:", save_path)

    return sub_out, wide_pred

# <<< ADDED: 찾은 최적 파라미터로 최종 모델 학습 및 제출

# 1. Optuna가 찾은 최적의 파라미터를 분리
best_params = study.best_params
best_clf_params = {k.replace('clf_', ''): v for k, v in best_params.items() if k.startswith('clf_')}
best_reg_params = {k.replace('reg_', ''): v for k, v in best_params.items() if k.startswith('reg_')}
best_post_params = {k: v for k, v in best_params.items() if k in ['alpha']} # 필요시 다른 후처리 파라미터 추가

# 2. 전체 학습 데이터 준비 (기존 코드와 유사)
tr_feat_full = add_ts_features(train_df)
tr_feat_full = add_days_since_last_sale(tr_feat_full)
Xf_full = tr_feat_full[feature_cols].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float)
y_full  = pd.to_numeric(tr_feat_full['sales_amount'], errors='coerce').fillna(0).clip(lower=0).to_numpy()

# 3. 최적 파라미터로 최종 모델 학습
# 분류기
pos_rate = (y_full > 0).mean(); neg_pos  = (1 - pos_rate) / max(pos_rate, 1e-6)
clf = XGBClassifier(**best_clf_params, random_state=42, n_jobs=-1, scale_pos_weight=neg_pos)
clf.fit(Xf_full, (y_full > 0).astype(int))

# 회귀기
w_full = 1.0 / (1.0 + y_full)
reg = XGBRegressor(**best_reg_params, random_state=42, n_jobs=-1)
reg.fit(Xf_full, np.log1p(y_full), sample_weight=w_full)

# 4. 최적 후처리 파라미터로 최종 예측
gate_on_optimized = dict(
    gate_mode='soft',
    alpha=best_post_params['alpha'], # Optuna가 찾은 alpha 값
    # 기존 사용하던 다른 후처리 값들
    tau_floor_prob=0.00,
    floor_ratio=0.15,
    scale_clip=(0.8, 12.0),
    prob_gamma=0.60,
    apply_scale=True,
    apply_floor=True
)

full_pred_df = infer_all_blocks(
    './test/TEST_*.csv',
    train_df, feature_cols, clf, reg,
    gate_on_optimized
)

# 5. 제출 파일 생성
sub_out, _ = build_submission(full_pred_df, './sample_submission.csv', save_path='./submission/submission_optuna_final.csv')
print("✅ Optuna 최적화 최종 제출 파일 생성 완료!")



[SUB] sum: 127050.0
[SUB] nonzero: 11801
[SUB] zero_ratio: 0.12649888971132495
💾 Saved: ./submission/submission_optuna_final.csv
✅ Optuna 최적화 최종 제출 파일 생성 완료!


In [ ]:
# 제출 빌드 뒤
vals = sub_out.iloc[:,1:].to_numpy(dtype='float64')
print("sum:", float(np.nansum(vals)))
print("nonzero:", int(np.count_nonzero(vals)))
print("zero_ratio:", float((vals == 0).mean()))

totals = sub_out.iloc[:,1:].sum(axis=1)
print("day total min/mean/max:", totals.min(), totals.mean(), totals.max())
print(sub_out.iloc[:,1:].sum(axis=0).sort_values(ascending=False).head(10))


sum: 126685.25967966193
nonzero: 12508
zero_ratio: 0.07416728349370837
day total min/mean/max: 274.49999909475383 1809.7894239951702 4172.999999999999
영업장명_메뉴명
화담숲주막_해물파전               8549.745043
미라시아_브런치(대인) 주말          5692.735080
포레스트릿_꼬치어묵               5486.466318
카페테리아_단체식 18000(신)       5330.762730
카페테리아_단체식 13000(신)       4415.622385
연회장_Cass Beer            3450.350925
포레스트릿_생수                 3351.568533
포레스트릿_떡볶이                3272.940247
연회장_Regular Coffee       3023.320721
미라시아_(단체)브런치주중 36,000    2956.387488
dtype: float64
